In [59]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [60]:
df = pd.read_csv("trainingData.csv")
df = df.drop('Unnamed: 0', axis=1)

In [61]:
df

,displacementX,displacementY,displacement,totalTravelX,totalTravelY,totalTravel,timeElapsed,meanStepDisplacementX,meanStepDisplacementY,meanStepDisplacement,...,label,meanVideoStepDisplacementX,meanVideoStepDisplacementY,meanVideoStepDisplacement,sdVideoMeanStepDisplacementX,sdVideoMeanStepDisplacementY,sdVideoMeanStepDisplacement,sdVideoMeanStepTravelX,sdVideoMeanStepTravelY,sdVideoMeanStepTravel
0,-37.353,-15.415,40.408772,40.603,24.497,47.420529,301.0,-0.124096,-0.051213,0.134248,...,0.0,0.017546,-0.295327,0.615731,0.283373,1.072575,0.969082,0.645286,0.949995,1.132996
1,-27.248,-18.540,32.957322,33.082,28.292,43.529944,301.0,-0.090525,-0.061595,0.109493,...,0.0,0.017546,-0.295327,0.615731,0.283373,1.072575,0.969082,0.645286,0.949995,1.132996
2,-29.438,-11.623,31.649486,38.020,26.041,46.083121,301.0,-0.097801,-0.038615,0.105148,...,0.0,0.017546,-0.295327,0.615731,0.283373,1.072575,0.969082,0.645286,0.949995,1.132996
3,-39.837,-14.879,42.524948,46.421,34.359,57.753356,301.0,-0.132349,-0.049432,0.141279,...,0.0,0.017546,-0.295327,0.615731,0.283373,1.072575,0.969082,0.645286,0.949995,1.132996
4,-1.616,0.055,1.616936,17.134,20.303,26.566629,301.0,-0.005369,0.000183,0.005372,...,0.0,0.017546,-0.295327,0.615731,0.283373,1.072575,0.969082,0.645286,0.949995,1.132996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16064,201.000,37.000,204.377102,557.000,509.000,754.539595,300.0,0.667774,0.122924,0.678994,...,1.0,0.893886,1.001751,1.510725,0.730266,0.577092,0.621749,0.506565,0.540807,0.688504
16065,530.000,137.000,547.420314,548.000,311.000,630.099199,181.0,2.912088,0.752747,3.007804,...,1.0,0.893886,1.001751,1.510725,0.730266,0.577092,0.621749,0.506565,0.540807,0.688504
16066,2.000,140.000,140.014285,564.000,602.000,824.924239,300.0,0.006645,0.465116,0.465164,...,1.0,0.893886,1.001751,1.510725,0.730266,0.577092,0.621749,0.506565,0.540807,0.688504
16067,21.000,24.000,31.890437,21.000,24.000,31.890437,12.0,1.615385,1.846154,2.453111,...,1.0,0.893886,1.001751,1.510725,0.730266,0.577092,0.621749,0.506565,0.540807,0.688504


In [62]:
df.sample(frac=1)
trainsize = int(len(df.index)*0.8)
df_train = df.iloc[:trainsize,:]
df_test = df.iloc[trainsize:,:]

In [63]:
df_train

,displacementX,displacementY,displacement,totalTravelX,totalTravelY,totalTravel,timeElapsed,meanStepDisplacementX,meanStepDisplacementY,meanStepDisplacement,...,label,meanVideoStepDisplacementX,meanVideoStepDisplacementY,meanVideoStepDisplacement,sdVideoMeanStepDisplacementX,sdVideoMeanStepDisplacementY,sdVideoMeanStepDisplacement,sdVideoMeanStepTravelX,sdVideoMeanStepTravelY,sdVideoMeanStepTravel
0,-37.353,-15.415,40.408772,40.603,24.497,47.420529,301.0,-0.124096,-0.051213,0.134248,...,0.0,0.017546,-0.295327,0.615731,0.283373,1.072575,0.969082,0.645286,0.949995,1.132996
1,-27.248,-18.540,32.957322,33.082,28.292,43.529944,301.0,-0.090525,-0.061595,0.109493,...,0.0,0.017546,-0.295327,0.615731,0.283373,1.072575,0.969082,0.645286,0.949995,1.132996
2,-29.438,-11.623,31.649486,38.020,26.041,46.083121,301.0,-0.097801,-0.038615,0.105148,...,0.0,0.017546,-0.295327,0.615731,0.283373,1.072575,0.969082,0.645286,0.949995,1.132996
3,-39.837,-14.879,42.524948,46.421,34.359,57.753356,301.0,-0.132349,-0.049432,0.141279,...,0.0,0.017546,-0.295327,0.615731,0.283373,1.072575,0.969082,0.645286,0.949995,1.132996
4,-1.616,0.055,1.616936,17.134,20.303,26.566629,301.0,-0.005369,0.000183,0.005372,...,0.0,0.017546,-0.295327,0.615731,0.283373,1.072575,0.969082,0.645286,0.949995,1.132996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12850,269.000,282.000,389.724261,269.000,282.000,389.724261,273.0,0.981752,1.029197,1.422351,...,0.0,0.915053,1.108350,1.565864,0.603175,0.549367,0.528635,0.485427,0.570013,0.698653
12851,227.000,211.000,309.919344,227.000,211.000,309.919344,216.0,1.046083,0.972350,1.428200,...,0.0,0.915053,1.108350,1.565864,0.603175,0.549367,0.528635,0.485427,0.570013,0.698653
12852,302.000,305.000,429.219058,302.000,305.000,429.219058,300.0,1.003322,1.013289,1.425977,...,0.0,0.915053,1.108350,1.565864,0.603175,0.549367,0.528635,0.485427,0.570013,0.698653
12853,289.000,303.000,418.724253,289.000,303.000,418.724253,289.0,0.996552,1.044828,1.443877,...,0.0,0.915053,1.108350,1.565864,0.603175,0.549367,0.528635,0.485427,0.570013,0.698653


In [65]:
# Normalizing data
NPTX = df_train.drop('label', axis=1).values
scaler = StandardScaler()
scaler.fit(NPTX)
NPTX=scaler.transform(NPTX)
NPTY = df_test.drop('label', axis=1).values
NPTY=scaler.transform(NPTY)

In [66]:
# Convert data to torch tensors
train_X = torch.tensor(NPTX).to(torch.float32)
train_y = torch.tensor(pd.DataFrame(df_train['label']).values).to(torch.float32)

test_X = torch.tensor(NPTY).to(torch.float32)
test_y = torch.tensor(pd.DataFrame(df_test['label']).values).to(torch.float32)

test_x=scaler.transform(test_X)


# Make datasets and dataloaders

train_dataset = TensorDataset(train_X, train_y)
test_dataset = TensorDataset(test_X, test_y)


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True) 

In [67]:
NPTX

array([[-1.42019864, -1.45462554, -1.42661118, ...,  0.25667883,
         2.09605826,  1.53637138],
       [-1.35362611, -1.47673125, -1.46999625, ...,  0.25667883,
         2.09605826,  1.53637138],
       [-1.368054  , -1.42780157, -1.47761095, ...,  0.25667883,
         2.09605826,  1.53637138],
       ...,
       [ 0.81548527,  0.81193556,  0.83718466, ..., -0.46813495,
         0.11377497, -0.20296915],
       [ 0.72984027,  0.7977879 ,  0.77608006, ..., -0.46813495,
         0.11377497, -0.20296915],
       [ 0.57831448,  0.56435152,  0.54490613, ..., -0.46813495,
         0.11377497, -0.20296915]])

In [68]:
# Model building- the fun stuff

model = nn.Sequential(
    nn.Flatten(), 
    nn.Linear(29, 5),
    nn.Linear(5, 1),
    nn.Sigmoid() # 
)

# model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.BCELoss() # Should modify to F2

In [70]:
# Some layers, such as Dropout, behave differently during training
model.train()
epochs = 20

for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        # Erase accumulated gradients
        optimizer.zero_grad()

        # Forward pass
        output = model(data)

        # Calculate loss
        loss = loss_fn(output, target)

        # Backward pass
        loss.backward()
        
        # Weight update
        optimizer.step()

    # Track loss each epoch
    print('Train Epoch: %d  Loss: %.4f' % (epoch + 1,  loss.item()))

Train Epoch: 1  Loss: 0.0091
Train Epoch: 2  Loss: 0.0394
Train Epoch: 3  Loss: 0.0097
Train Epoch: 4  Loss: 0.0062
Train Epoch: 5  Loss: 0.0238
Train Epoch: 6  Loss: 0.0879
Train Epoch: 7  Loss: 0.0387
Train Epoch: 8  Loss: 0.3097
Train Epoch: 9  Loss: 0.0300
Train Epoch: 10  Loss: 0.0054
Train Epoch: 11  Loss: 0.0073
Train Epoch: 12  Loss: 0.0164
Train Epoch: 13  Loss: 0.0088
Train Epoch: 14  Loss: 0.0107
Train Epoch: 15  Loss: 0.3318
Train Epoch: 16  Loss: 0.0373
Train Epoch: 17  Loss: 0.0063
Train Epoch: 18  Loss: 0.0017
Train Epoch: 19  Loss: 0.0144
Train Epoch: 20  Loss: 0.0054


In [72]:
# Testing
model.eval()

test_loss = 0
correct = 0


with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        test_loss += loss_fn(output, target).item()  # Sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max class score
        correct += pred.eq(target.view_as(pred)).sum().item()


test_loss /= len(test_loader.dataset)

print('Test set: Average loss: %.4f, Accuracy: %d/%d (%.4f)' %
      (test_loss, correct, len(test_loader.dataset),
       100. * correct / len(test_loader.dataset)))

Test set: Average loss: 0.0019, Accuracy: 1605/3214 (49.9378)
